# 5.2.Custom_Model_VGG19_gray

In [ ]:
#!pip install sklearn
#!pip install imutils

In [20]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD, Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from imutils import paths

from sklearn.preprocessing import LabelBinarizer, LabelEncoder

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


# Carga de los datos para el entrenamiento

In [36]:
img_size = 224
batch_size = 8
epochs = 10
path_checkpoints = 'checkpoint_VGG19'

path_train = 'data_' + str(img_size) + '/train'
path_validation = 'data_' + str(img_size) + '/validation'

In [37]:
imagePaths_train = list(paths.list_images(path_train))
imagePaths_validation = list(paths.list_images(path_validation))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths_train]
classNames = [str(x) for x in np.unique(classNames)]

print("etiquetas del entrenamiento")
print(classNames)

#Códificando las etiquetas en númerops
print("codificando etiquetas")
le = LabelEncoder()
classNames_ids = le.fit_transform(classNames)
print(classNames_ids)

etiquetas del entrenamiento
['10', '100', '20', '200', '50']
codificando etiquetas
[0 1 2 3 4]


In [38]:
aug_train = ImageDataGenerator(rescale=1. / 255)
#aug_train.fit(data_train)
train_generator = aug_train.flow_from_directory(path_train,
                                                color_mode='grayscale',
                                                target_size=(img_size, img_size),
                                                batch_size=batch_size)

Found 59812 images belonging to 5 classes.


In [39]:
aug_validation = ImageDataGenerator(rescale=1. / 255)
#aug_validation.fit(data_validation)
validation_generator = aug_validation.flow_from_directory(path_validation,
                                                color_mode='grayscale',
                                                          target_size=(img_size, img_size),
                                                          batch_size=batch_size)

Found 3028 images belonging to 5 classes.


# Creamos el modelo tipo VGG19

In [40]:
model = Sequential()

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',input_shape=(img_size, img_size, 1)))
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
#110

#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
#53

#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, kernel_size=(4, 4), activation='relu'))
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
#22

#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu'))
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu'))
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu'))
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
#7

#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(2048, kernel_size=(3, 3), activation='relu'))
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(2048, kernel_size=(3, 3), activation='relu'))
#model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(2048, kernel_size=(3, 3), activation='relu'))
#model.add(ZeroPadding2D((1,1)))
#model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
#model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(classNames), activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 222, 222, 128)     1280      
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 220, 220, 128)     147584    
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 110, 110, 128)     0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 108, 108, 256)     295168    
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 106, 106, 256)     590080    
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 53, 53, 256)       0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 50, 50, 512)      

## Compilamos el modelo

In [41]:
#default learning_rate = 0.001
#optimizer = SGD(learning_rate=0.0001)
optimizer = Adam(learning_rate=0.0001)
loss = 'categorical_crossentropy'
metrics = ['accuracy']

model.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=metrics)

In [42]:
fname = os.path.sep.join([path_checkpoints, 
                          'epoch-{epoch:03d}-val_loss:{val_loss:.4f}-val_accuracy:{val_accuracy:.4f}.hdf5'])
checkpoint = ModelCheckpoint(fname,
                             monitor='val_accuracy',
                             save_best_only=True,
                             mode='max',
                             verbose=1)
callbacks = [checkpoint]
#callbacks = []

In [43]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=len(train_generator),
                              validation_data=validation_generator,
                              epochs = epochs,
                              callbacks=callbacks,
                              verbose = 1)

Epoch 1/10
7477/7477 [==============================] - ETA: 0s - loss: 1.6098 - accuracy: 0.1987
Epoch 00001: val_accuracy improved from -inf to 0.19980, saving model to checkpoint_VGG19/epoch-001-val_loss:1.6098-val_accuracy:0.1998.hdf5
7477/7477 [==============================] - 2460s 329ms/step - loss: 1.6098 - accuracy: 0.1987 - val_loss: 1.6098 - val_accuracy: 0.1998
Epoch 2/10
7477/7477 [==============================] - ETA: 0s - loss: 1.6097 - accuracy: 0.1982
Epoch 00002: val_accuracy did not improve from 0.19980
7477/7477 [==============================] - 2446s 327ms/step - loss: 1.6097 - accuracy: 0.1982 - val_loss: 1.6095 - val_accuracy: 0.1998
Epoch 3/10
7477/7477 [==============================] - ETA: 0s - loss: 1.6096 - accuracy: 0.1990
Epoch 00003: val_accuracy improved from 0.19980 to 0.20112, saving model to checkpoint_VGG19/epoch-003-val_loss:1.6095-val_accuracy:0.2011.hdf5
7477/7477 [==============================] - 2452s 328ms/step - loss: 1.6096 - accuracy: 0

KeyboardInterrupt: 

## Creando un gráfico para la revisión

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.ylim([0,1.1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.ylim([0,12])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Guardando el modelo

In [ ]:
saved_model_dir = 'model'
tf.saved_model.save(model, saved_model_dir)

## Convirtiendo el modelo en tensorflow lite para el uso en mobile

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

path_tensorflow_lite = 'model_tflite/vgg19_model.tflite'
with open(path_tensorflow_lite, 'wb') as f:
  f.write(tflite_model)

# Tomando 25 imágenes de validación y observando el resultado

In [ ]:
test_data = ImageClassifierDataLoader.from_folder(path_validation)

def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

plt.figure(figsize=(20, 20))

for i, (image, label) in enumerate(test_data.dataset.take(100)):
    ax = plt.subplot(10, 10, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image.numpy(), cmap=plt.cm.gray)
    
    # predict
    swapped = np.moveaxis(image, 0, 1)
    arr4d = np.expand_dims(swapped, 0)
    id_predict = np.argmax(model.predict(arr4d))
    predict_label = classNames[id_predict]
    
    color = get_label_color(predict_label, test_data.index_to_label[label.numpy()])
    #color = 'black'
    ax.xaxis.label.set_color(color)
    plt.xlabel('Predicted: %s' % predict_label)
plt.show()